In [1]:
# Import libraries
import awswrangler as wr
import numpy as np
import pandas as pd
#%%
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np
from sklearn.metrics import mean_absolute_error
print(tf.__version__)
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

2021-12-14 00:58:07.727729: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-14 00:58:07.727758: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.6.0


In [2]:
# read the files 
meth = pd.read_csv('methyl_merged_subset.csv') # methylation data

In [3]:
meth.head()

,meth_id,meth_id.1,Endometriosis,Cycle_Phase,Stage,Institute,Batch,cg21870274,cg09499020,cg16535257,...,cg23957653,cg03975652,cg25500853,cg04292080,cg17456675,cg02492207,cg10994397,cg06002660,cg25900627,cg04854189
0,201904640089_R01C01,201904640089_R01C01,Yes,PE,III-IV,"IMB, UQ",1,2.039036,-0.721799,-0.411768,...,3.797159,0.920605,3.824859,3.763009,3.253624,1.492936,3.390775,2.924680,3.066238,3.895433
1,201904640089_R02C01,201904640089_R02C01,No,PE,CONTROL,UCSF,1,1.960953,-0.483177,-0.821167,...,3.511649,1.173255,3.668846,3.377942,2.874201,1.730391,2.822060,2.711121,3.508393,4.200534
2,201904640089_R03C01,201904640089_R03C01,Yes,SE,III-IV,UCSF,1,2.091686,-0.779545,-1.158785,...,5.070362,0.236193,3.871971,3.306408,3.056062,1.529067,2.074006,2.325743,3.114843,3.228607
3,201904640089_R04C01,201904640089_R04C01,Yes,SE,I-II,"IMB, UQ",1,2.681174,-0.641505,-0.856451,...,3.727402,0.959746,3.698094,3.173313,3.026365,1.590253,2.994565,2.912144,3.161728,3.593516
4,201904640089_R05C01,201904640089_R05C01,No,PE,CONTROL,"IMB, UQ",1,1.765793,-1.132911,-0.136729,...,3.367666,1.174925,3.908117,3.079902,2.785387,1.198299,2.781944,2.955136,2.778078,3.067055


In [4]:
meth = meth.rename(columns={'Unnamed: 0': 'probe'})
meth = meth.set_index('meth_id')
meth.drop(columns = ['meth_id.1'], inplace = True)
#meth = meth.transpose()

In [5]:
meth

,Endometriosis,Cycle_Phase,Stage,Institute,Batch,cg21870274,cg09499020,cg16535257,cg06325811,cg08258224,...,cg23957653,cg03975652,cg25500853,cg04292080,cg17456675,cg02492207,cg10994397,cg06002660,cg25900627,cg04854189
meth_id,,,,,,,,,,,,,,,,,,,,,
201904640089_R01C01,Yes,PE,III-IV,"IMB, UQ",1,2.039036,-0.721799,-0.411768,-1.484742,3.083376,...,3.797159,0.920605,3.824859,3.763009,3.253624,1.492936,3.390775,2.924680,3.066238,3.895433
201904640089_R02C01,No,PE,CONTROL,UCSF,1,1.960953,-0.483177,-0.821167,-1.421923,3.035740,...,3.511649,1.173255,3.668846,3.377942,2.874201,1.730391,2.822060,2.711121,3.508393,4.200534
201904640089_R03C01,Yes,SE,III-IV,UCSF,1,2.091686,-0.779545,-1.158785,-1.266587,2.730783,...,5.070362,0.236193,3.871971,3.306408,3.056062,1.529067,2.074006,2.325743,3.114843,3.228607
201904640089_R04C01,Yes,SE,I-II,"IMB, UQ",1,2.681174,-0.641505,-0.856451,-1.487919,2.934617,...,3.727402,0.959746,3.698094,3.173313,3.026365,1.590253,2.994565,2.912144,3.161728,3.593516
201904640089_R05C01,No,PE,CONTROL,"IMB, UQ",1,1.765793,-1.132911,-0.136729,-1.686892,3.200332,...,3.367666,1.174925,3.908117,3.079902,2.785387,1.198299,2.781944,2.955136,2.778078,3.067055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202852840153_R04C01,Yes,PE,III-IV,UCSF,2,1.937107,-0.593792,-0.388072,-1.541838,3.206347,...,3.686830,0.911204,3.451644,3.373164,2.881161,1.226894,2.930886,2.801135,3.239445,3.481191
202852840153_R07C01,Yes,SE,III-IV,UCSF,2,2.081493,-0.565563,-0.506641,-1.384473,2.584101,...,4.083370,0.906261,3.920661,3.207855,2.907179,1.194733,2.790707,2.422487,2.650287,3.301067
202184900058_R08C01,Yes,Menstrual,I-II,Oxford,1,1.885567,-0.764100,-0.694129,-1.369279,2.843320,...,3.134838,0.924110,3.480679,3.897287,2.900470,1.201403,2.755273,2.528899,2.739883,3.181087


In [6]:
# Change Endo Yes/No to binary
meth['Endometriosis_int'] = np.where(meth['Endometriosis'] == 'Yes', 1, meth['Endometriosis'])
meth['Endometriosis_int'] = np.where(meth['Endometriosis_int'] == 'No', 0, meth['Endometriosis_int'])

In [7]:
meth

,Endometriosis,Cycle_Phase,Stage,Institute,Batch,cg21870274,cg09499020,cg16535257,cg06325811,cg08258224,...,cg03975652,cg25500853,cg04292080,cg17456675,cg02492207,cg10994397,cg06002660,cg25900627,cg04854189,Endometriosis_int
meth_id,,,,,,,,,,,,,,,,,,,,,
201904640089_R01C01,Yes,PE,III-IV,"IMB, UQ",1,2.039036,-0.721799,-0.411768,-1.484742,3.083376,...,0.920605,3.824859,3.763009,3.253624,1.492936,3.390775,2.924680,3.066238,3.895433,1
201904640089_R02C01,No,PE,CONTROL,UCSF,1,1.960953,-0.483177,-0.821167,-1.421923,3.035740,...,1.173255,3.668846,3.377942,2.874201,1.730391,2.822060,2.711121,3.508393,4.200534,0
201904640089_R03C01,Yes,SE,III-IV,UCSF,1,2.091686,-0.779545,-1.158785,-1.266587,2.730783,...,0.236193,3.871971,3.306408,3.056062,1.529067,2.074006,2.325743,3.114843,3.228607,1
201904640089_R04C01,Yes,SE,I-II,"IMB, UQ",1,2.681174,-0.641505,-0.856451,-1.487919,2.934617,...,0.959746,3.698094,3.173313,3.026365,1.590253,2.994565,2.912144,3.161728,3.593516,1
201904640089_R05C01,No,PE,CONTROL,"IMB, UQ",1,1.765793,-1.132911,-0.136729,-1.686892,3.200332,...,1.174925,3.908117,3.079902,2.785387,1.198299,2.781944,2.955136,2.778078,3.067055,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202852840153_R04C01,Yes,PE,III-IV,UCSF,2,1.937107,-0.593792,-0.388072,-1.541838,3.206347,...,0.911204,3.451644,3.373164,2.881161,1.226894,2.930886,2.801135,3.239445,3.481191,1
202852840153_R07C01,Yes,SE,III-IV,UCSF,2,2.081493,-0.565563,-0.506641,-1.384473,2.584101,...,0.906261,3.920661,3.207855,2.907179,1.194733,2.790707,2.422487,2.650287,3.301067,1
202184900058_R08C01,Yes,Menstrual,I-II,Oxford,1,1.885567,-0.764100,-0.694129,-1.369279,2.843320,...,0.924110,3.480679,3.897287,2.900470,1.201403,2.755273,2.528899,2.739883,3.181087,1


In [8]:
# Split dataset into features and labels
x = meth.iloc[:, 5:999] # CPG probes
y = meth.iloc[:, 999] # Endo Case vs Control 

In [9]:
# Split data into train and test (80/20) stratified by cycle phase 
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify=meth['Cycle_Phase'], test_size=0.2)

In [10]:
# Change labels to int for linear regression 
y_train = y_train.astype('float64')

# Transpose features df for lin regress
X_train = X_train.T

In [11]:
# set up function for feature extraction using linear regression
import scipy 
from scipy.stats import linregress
def calc_slope(row):
    a = scipy.stats.linregress(y_train, row)
    #print(pd.Series(a._asdict()))
    return pd.Series(a._asdict())

cols = X_train.columns

X_train = X_train[cols].apply(pd.to_numeric, errors='coerce').fillna(0)

In [12]:
# Perform feature extraction
feat_extract = X_train.join(X_train.apply(calc_slope,axis=1))
feat_extract

,202184900058_R06C01,202229250043_R05C01,202835130042_R05C01,202878330078_R03C01,201985320094_R05C01,202835130080_R06C01,202184900058_R02C01,202852840047_R08C01,202212330069_R04C01,202212330091_R02C01,...,202878330112_R02C01,202878330113_R05C01,202176300030_R04C01,202013790112_R06C01,slope,intercept,rvalue,pvalue,stderr,intercept_stderr
cg21870274,1.926250,1.580234,2.717510,2.604533,1.950098,2.214069,1.794157,1.735872,1.788965,2.262396,...,2.335508,2.526619,2.251331,1.665795,-0.022355,2.032825,-0.031585,0.372296,0.025042,0.020190
cg09499020,-0.856607,-0.780043,-0.943705,-0.771446,-1.047634,-0.502149,-0.623979,-0.937581,-0.943179,-0.935765,...,-0.767764,-0.873174,-0.684609,-0.857367,-0.007402,-0.751770,-0.014400,0.684236,0.018194,0.014668
cg16535257,-1.219402,-0.242571,-0.785318,-0.400548,-0.597603,-0.393439,-0.486783,-0.783721,-0.922747,-0.369934,...,-0.315533,-0.595121,-0.637735,-0.773836,-0.059523,-0.591877,-0.079716,0.024148,0.026348,0.021243
cg06325811,-1.608578,-1.633142,-1.431429,-1.467665,-1.561937,-1.460820,-1.279057,-1.425017,-1.409107,-1.167399,...,-1.424057,-1.617496,-1.611773,-2.842010,0.028115,-1.487184,0.052498,0.137922,0.018932,0.015263
cg08258224,2.503736,3.176005,3.006067,2.545781,3.180907,2.818270,2.855193,2.906057,3.067520,2.424159,...,3.030037,2.937856,2.339067,3.039344,-0.013510,2.868458,-0.021924,0.535772,0.021808,0.017582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg02492207,1.485185,0.902021,0.887848,1.218600,1.419742,1.474089,1.186891,1.350169,1.316898,1.250479,...,1.120298,0.952213,1.196504,1.899489,0.039839,1.273545,0.056270,0.111762,0.025023,0.020174
cg10994397,2.890773,2.718886,3.349877,2.599802,2.740962,2.981902,2.757630,3.094246,2.811005,2.885245,...,3.209177,2.808723,3.012197,2.210876,0.017534,2.835498,0.025693,0.468029,0.024151,0.019471
cg06002660,2.905867,3.062622,3.005932,2.903464,2.473774,2.806677,3.064245,2.939793,2.041185,2.913499,...,2.628761,2.675371,2.796033,3.782611,-0.006325,2.658904,-0.007123,0.840576,0.031432,0.025341
cg25900627,3.350774,2.934105,2.977458,3.348894,2.558821,2.979242,3.063618,2.983815,2.697074,2.451299,...,2.792513,2.837534,2.671976,3.444476,0.022492,2.897270,0.033229,0.347915,0.023948,0.019308


In [13]:
#Benjamini-Hochberg filtering p-values
feat_extract = feat_extract.sort_values(by='pvalue')
feat_extract['rank'] = feat_extract.reset_index().index + 1
feat_extract['(I/m)Q'] = (feat_extract['rank']/len(feat_extract))*0.05
feat_extract['(I/m)Q - p'] = feat_extract['(I/m)Q'] - feat_extract['pvalue']
feat_extract['nominally_sig'] = np.where(feat_extract['pvalue'] < 0.05, 1, 0)
feat_extract['BH_sig'] = np.where(feat_extract['(I/m)Q - p'] < 0, 0, 1)

In [14]:
feat_extract['nominally_sig'].sum(), feat_extract['BH_sig'].sum()

(98, 0)

In [15]:
# Filter for nominally significant probes
feat_extract_NomSig = feat_extract[feat_extract['nominally_sig'] == 1]

In [16]:
# remove output from linear regression
feat_extract_NomSig = feat_extract_NomSig.iloc[:,:800] 

In [17]:
feat_extract_NomSig = feat_extract_NomSig.T

In [18]:
feat_extract_NomSig

,cg16026189,cg01508988,cg08899471,cg13897348,cg27477059,cg27302190,cg20426768,cg17142931,cg04972348,cg04195702,...,cg04559824,cg19861048,cg21779088,cg01922485,cg20435334,cg19413397,cg16045223,cg15105996,cg25973293,cg19092396
202184900058_R06C01,3.999850,-2.839540,2.001039,-2.790806,3.036225,2.143516,0.919224,0.252282,0.024390,4.128234,...,2.170693,1.953526,4.639270,5.184312,2.992010,2.202932,0.715139,-0.007355,5.029478,-2.660539
202229250043_R05C01,3.950770,-2.378813,2.240852,-2.954704,2.704513,1.474531,0.651645,-0.079177,0.321233,4.512012,...,2.288927,1.805602,3.953061,4.739814,2.701177,2.320819,1.290945,-0.080173,4.304394,-2.970099
202835130042_R05C01,3.437832,-2.758592,2.139899,-2.823325,2.969324,2.085322,0.997941,0.076489,0.506988,4.214040,...,2.306523,1.853160,4.021908,5.044741,3.012121,2.027369,1.369375,0.101884,5.096871,-2.350843
202878330078_R03C01,2.420163,-2.528663,1.562888,-1.974657,2.926104,2.138184,0.628406,0.065944,0.411159,3.684718,...,2.656391,2.037115,3.859788,4.760932,2.746935,2.931939,0.703797,-0.145170,5.233263,-3.195751
201985320094_R05C01,3.413458,-2.563598,2.160956,-1.804074,2.563266,2.120897,0.880403,0.024712,0.239181,4.445891,...,2.456700,1.654236,4.335523,4.955000,3.078684,2.177536,0.073913,0.191024,4.959508,-2.791019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201904640205_R06C01,3.599834,-2.583206,2.097334,-2.322379,3.099335,2.076351,0.958917,-0.215553,0.290152,4.181092,...,2.395430,1.672504,3.990722,4.732229,2.472473,2.244349,0.606351,-0.071320,3.924035,-2.131066
202878330112_R02C01,3.670592,-2.611249,2.435342,-2.742501,3.023715,2.318759,0.774592,-0.081051,0.236751,4.222629,...,2.253329,1.901001,3.377173,4.961262,2.725664,2.645324,1.009227,-0.052745,5.326494,-2.168397
202878330113_R05C01,3.800672,-2.730738,2.521438,-2.119614,3.076722,1.433997,0.948820,0.034934,0.259330,4.826382,...,2.282470,1.632305,4.518949,4.957544,2.673847,2.411982,0.521026,0.019522,5.110144,-2.695156
202176300030_R04C01,3.989571,-2.569906,2.959877,-2.271660,2.950497,1.970687,0.993190,-0.021852,0.297660,4.205970,...,2.306185,1.903472,4.157781,5.057137,2.818165,2.692087,0.481266,0.067410,4.538736,-1.900375


In [19]:
pheno_path = 's3://adisiddharth/phenotype_mappings.csv'
pheno = wr.s3.read_csv(pheno_path)

In [20]:
pheno.head()

,Unnamed: 0,Study.ID,Sample.ID.for.EPIC.Array...2,Sample.ID,Alternative.Sample.ID.if.applicable,Shipping.Institute,Sample.Originating.Institute,DNA.extracting.Institute,Institute.for.Analysis,Control..Yes.No.,...,Affy..CEL.ID,Affy.CEL.Repeats,Cycle.Phase.Source,Prior.genotyping..Platform..1...UK.Biobank.Axiom.Array..2...Illumina.Infinium.Global.Screening.Array..3...Illumina.Infinium.PsychArray..4...Illumina.Infinium.CoreExome..0...Not.genotyped.,Genotyping.PLATE.No.,Genotyping.Plate.coordinates,Sample.ID.for.EPIC.Array...29,Batch,Genetic.Ancestry,inst_id
0,201904640089_R01C01,1.0,1,210274,NaN,"IMB, UQ",Uni of Melbourne,"IMB, UQ","IMB, UQ",No,...,Plate_1_A01_210274.CEL,NaN,Pathology,4.0,1,A1,1,1,EUR,"IMB, UQ_210274"
1,201904640089_R02C01,2.0,2,UC-236,1983,UCSF,UCSF,UCSF,UCSF,Yes,...,Plate_1_A02_UC-236.CEL,NaN,Hist,NaN,1,A2,2,1,EUR,UCSF_UC-236
2,201904640089_R03C01,3.0,3,CN-113,2565,UCSF,UCSF,UCSF,UCSF,No,...,Plate_1_A03_CN-113.CEL,NaN,Hist,NaN,1,A3,3,1,EUR,UCSF_CN-113
3,201904640089_R04C01,4.0,4,210373,NaN,"IMB, UQ",Uni of Melbourne,"IMB, UQ","IMB, UQ",No,...,Plate_1_A04_210373.CEL,NaN,Pathology,4.0,1,A4,4,1,ADMIX,"IMB, UQ_210373"
4,201904640089_R05C01,5.0,5,210394,NaN,"IMB, UQ",Uni of Melbourne,"IMB, UQ","IMB, UQ",Yes,...,Plate_1_A05_210394.CEL,NaN,Pathology,4.0,1,A5,5,1,AMR,"IMB, UQ_210394"


In [21]:
pheno = pheno.rename(columns={col: col.replace('.', '_') for col in pheno.columns})
pheno = pheno.rename(columns={'Unnamed: 0': 'meth_id'})
vars_of_interest = ['meth_id', 'Endometriosis__Yes_No_', 'Cycle_phase_for_Analysis', 'Endometriosis_stage_grouped__I_II____III_IV_', 'Institute_for_Analysis', 'Batch']
pheno = pheno.loc[:, vars_of_interest]
pheno = pheno.rename(columns={'Endometriosis__Yes_No_': 'Endometriosis', 'Endometriosis_stage_grouped__I_II____III_IV_': 'Stage', 'Institute_for_Analysis': 'Institute', 'Cycle_phase_for_Analysis': 'Cycle_Phase'})

In [22]:
pheno.head()

,meth_id,Endometriosis,Cycle_Phase,Stage,Institute,Batch
0,201904640089_R01C01,Yes,PE,III-IV,"IMB, UQ",1
1,201904640089_R02C01,No,PE,CONTROL,UCSF,1
2,201904640089_R03C01,Yes,ESE,III-IV,UCSF,1
3,201904640089_R04C01,Yes,MSE,I-II,"IMB, UQ",1
4,201904640089_R05C01,No,PE,CONTROL,"IMB, UQ",1


In [23]:
train_df = pheno.merge(feat_extract_NomSig, left_on='meth_id', right_on=feat_extract_NomSig.index, how='right')
train_df = train_df.loc[:, ~train_df.columns.duplicated()]

In [24]:
train_df

,meth_id,Endometriosis,Cycle_Phase,Stage,Institute,Batch,cg16026189,cg01508988,cg08899471,cg13897348,...,cg04559824,cg19861048,cg21779088,cg01922485,cg20435334,cg19413397,cg16045223,cg15105996,cg25973293,cg19092396
0,202184900058_R06C01,Yes,PE,I-II,UCSF,1,3.999850,-2.839540,2.001039,-2.790806,...,2.170693,1.953526,4.639270,5.184312,2.992010,2.202932,0.715139,-0.007355,5.029478,-2.660539
1,202229250043_R05C01,Yes,PE,I-II,UCSF,1,3.950770,-2.378813,2.240852,-2.954704,...,2.288927,1.805602,3.953061,4.739814,2.701177,2.320819,1.290945,-0.080173,4.304394,-2.970099
2,202835130042_R05C01,No,LSE,CONTROL,Oxford,2,3.437832,-2.758592,2.139899,-2.823325,...,2.306523,1.853160,4.021908,5.044741,3.012121,2.027369,1.369375,0.101884,5.096871,-2.350843
3,202878330078_R03C01,No,ESE,CONTROL,Oxford,2,2.420163,-2.528663,1.562888,-1.974657,...,2.656391,2.037115,3.859788,4.760932,2.746935,2.931939,0.703797,-0.145170,5.233263,-3.195751
4,201985320094_R05C01,No,MSE,CONTROL,UCSF,1,3.413458,-2.563598,2.160956,-1.804074,...,2.456700,1.654236,4.335523,4.955000,3.078684,2.177536,0.073913,0.191024,4.959508,-2.791019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,201904640205_R06C01,No,PE,CONTROL,Oxford,1,3.599834,-2.583206,2.097334,-2.322379,...,2.395430,1.672504,3.990722,4.732229,2.472473,2.244349,0.606351,-0.071320,3.924035,-2.131066
796,202878330112_R02C01,No,ESE,CONTROL,MAMC,2,3.670592,-2.611249,2.435342,-2.742501,...,2.253329,1.901001,3.377173,4.961262,2.725664,2.645324,1.009227,-0.052745,5.326494,-2.168397
797,202878330113_R05C01,Yes,LSE,I-II,MAMC,2,3.800672,-2.730738,2.521438,-2.119614,...,2.282470,1.632305,4.518949,4.957544,2.673847,2.411982,0.521026,0.019522,5.110144,-2.695156
798,202176300030_R04C01,Yes,LSE,III-IV,"IMB, UQ",1,3.989571,-2.569906,2.959877,-2.271660,...,2.306185,1.903472,4.157781,5.057137,2.818165,2.692087,0.481266,0.067410,4.538736,-1.900375


In [25]:
# Group ESE, MSE and LSE into SE 
train_df.loc[train_df['Cycle_Phase'].isin(['ESE', 'MSE', 'LSE']), 'Cycle_Phase'] = 'SE'
#merged_df['Cycle_Phase_Class'] = (merged_df['Cycle_Phase'].rank(method='dense') - 1).astype('int')
#merged_df['Stage_Class'] = (merged_df['Stage'].rank(method='dense') - 1).astype('int')
#merged_df['Endometriosis_Class'] = (merged_df['Endometriosis'].rank(method='dense') - 1).astype('int')
#merged_df['Batch_Class'] = (merged_df['Batch'].rank(method='dense') - 1).astype('int')
#merged_df['Institute_Class'] = (merged_df['Institute'].rank(method='dense') - 1).astype('int')

In [26]:
train_df

,meth_id,Endometriosis,Cycle_Phase,Stage,Institute,Batch,cg16026189,cg01508988,cg08899471,cg13897348,...,cg04559824,cg19861048,cg21779088,cg01922485,cg20435334,cg19413397,cg16045223,cg15105996,cg25973293,cg19092396
0,202184900058_R06C01,Yes,PE,I-II,UCSF,1,3.999850,-2.839540,2.001039,-2.790806,...,2.170693,1.953526,4.639270,5.184312,2.992010,2.202932,0.715139,-0.007355,5.029478,-2.660539
1,202229250043_R05C01,Yes,PE,I-II,UCSF,1,3.950770,-2.378813,2.240852,-2.954704,...,2.288927,1.805602,3.953061,4.739814,2.701177,2.320819,1.290945,-0.080173,4.304394,-2.970099
2,202835130042_R05C01,No,SE,CONTROL,Oxford,2,3.437832,-2.758592,2.139899,-2.823325,...,2.306523,1.853160,4.021908,5.044741,3.012121,2.027369,1.369375,0.101884,5.096871,-2.350843
3,202878330078_R03C01,No,SE,CONTROL,Oxford,2,2.420163,-2.528663,1.562888,-1.974657,...,2.656391,2.037115,3.859788,4.760932,2.746935,2.931939,0.703797,-0.145170,5.233263,-3.195751
4,201985320094_R05C01,No,SE,CONTROL,UCSF,1,3.413458,-2.563598,2.160956,-1.804074,...,2.456700,1.654236,4.335523,4.955000,3.078684,2.177536,0.073913,0.191024,4.959508,-2.791019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,201904640205_R06C01,No,PE,CONTROL,Oxford,1,3.599834,-2.583206,2.097334,-2.322379,...,2.395430,1.672504,3.990722,4.732229,2.472473,2.244349,0.606351,-0.071320,3.924035,-2.131066
796,202878330112_R02C01,No,SE,CONTROL,MAMC,2,3.670592,-2.611249,2.435342,-2.742501,...,2.253329,1.901001,3.377173,4.961262,2.725664,2.645324,1.009227,-0.052745,5.326494,-2.168397
797,202878330113_R05C01,Yes,SE,I-II,MAMC,2,3.800672,-2.730738,2.521438,-2.119614,...,2.282470,1.632305,4.518949,4.957544,2.673847,2.411982,0.521026,0.019522,5.110144,-2.695156
798,202176300030_R04C01,Yes,SE,III-IV,"IMB, UQ",1,3.989571,-2.569906,2.959877,-2.271660,...,2.306185,1.903472,4.157781,5.057137,2.818165,2.692087,0.481266,0.067410,4.538736,-1.900375


In [27]:
train_df.set_axis(train_df['meth_id'], inplace = True)

In [28]:
train_df

,meth_id,Endometriosis,Cycle_Phase,Stage,Institute,Batch,cg16026189,cg01508988,cg08899471,cg13897348,...,cg04559824,cg19861048,cg21779088,cg01922485,cg20435334,cg19413397,cg16045223,cg15105996,cg25973293,cg19092396
meth_id,,,,,,,,,,,,,,,,,,,,,
202184900058_R06C01,202184900058_R06C01,Yes,PE,I-II,UCSF,1,3.999850,-2.839540,2.001039,-2.790806,...,2.170693,1.953526,4.639270,5.184312,2.992010,2.202932,0.715139,-0.007355,5.029478,-2.660539
202229250043_R05C01,202229250043_R05C01,Yes,PE,I-II,UCSF,1,3.950770,-2.378813,2.240852,-2.954704,...,2.288927,1.805602,3.953061,4.739814,2.701177,2.320819,1.290945,-0.080173,4.304394,-2.970099
202835130042_R05C01,202835130042_R05C01,No,SE,CONTROL,Oxford,2,3.437832,-2.758592,2.139899,-2.823325,...,2.306523,1.853160,4.021908,5.044741,3.012121,2.027369,1.369375,0.101884,5.096871,-2.350843
202878330078_R03C01,202878330078_R03C01,No,SE,CONTROL,Oxford,2,2.420163,-2.528663,1.562888,-1.974657,...,2.656391,2.037115,3.859788,4.760932,2.746935,2.931939,0.703797,-0.145170,5.233263,-3.195751
201985320094_R05C01,201985320094_R05C01,No,SE,CONTROL,UCSF,1,3.413458,-2.563598,2.160956,-1.804074,...,2.456700,1.654236,4.335523,4.955000,3.078684,2.177536,0.073913,0.191024,4.959508,-2.791019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201904640205_R06C01,201904640205_R06C01,No,PE,CONTROL,Oxford,1,3.599834,-2.583206,2.097334,-2.322379,...,2.395430,1.672504,3.990722,4.732229,2.472473,2.244349,0.606351,-0.071320,3.924035,-2.131066
202878330112_R02C01,202878330112_R02C01,No,SE,CONTROL,MAMC,2,3.670592,-2.611249,2.435342,-2.742501,...,2.253329,1.901001,3.377173,4.961262,2.725664,2.645324,1.009227,-0.052745,5.326494,-2.168397
202878330113_R05C01,202878330113_R05C01,Yes,SE,I-II,MAMC,2,3.800672,-2.730738,2.521438,-2.119614,...,2.282470,1.632305,4.518949,4.957544,2.673847,2.411982,0.521026,0.019522,5.110144,-2.695156


In [29]:
# Export feature extracted train set as csv
train_df.to_csv('Endometriosis_subset_train_feature_extract.csv')

In [30]:
# Keep the same features as training set in test set
test = X_test[feat_extract_NomSig.columns]

In [31]:
# Add y_test (test labels) back to test features
test['Endometriosis_int'] = y_test
test

/tmp/ipykernel_5580/673509507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Endometriosis_int'] = y_test


,cg16026189,cg01508988,cg08899471,cg13897348,cg27477059,cg27302190,cg20426768,cg17142931,cg04972348,cg04195702,...,cg19861048,cg21779088,cg01922485,cg20435334,cg19413397,cg16045223,cg15105996,cg25973293,cg19092396,Endometriosis_int
meth_id,,,,,,,,,,,,,,,,,,,,,
202878330076_R05C01,3.097823,-2.539343,2.046007,-2.478576,2.563572,1.965384,0.801510,-0.090231,0.207092,3.796747,...,1.752006,3.604724,5.075303,2.698030,1.942857,0.706064,-0.162937,4.262236,-2.779906,1
202835130054_R08C01,3.938173,-2.340656,2.661644,-2.422570,3.155311,2.241019,0.850654,-0.079164,0.270008,4.485042,...,1.701143,4.203569,4.793561,2.800377,1.822075,1.073474,-0.141250,5.361843,-2.996784,1
202212330173_R07C01,3.345793,-2.804006,1.974256,-2.452216,2.525939,2.506842,0.983303,-0.018804,0.100521,3.766057,...,1.879855,4.217157,5.218384,2.731195,2.057755,1.265913,-0.090030,5.607344,-2.471398,0
202013790074_R03C01,3.969819,-2.698969,2.352478,-2.081186,2.999481,2.229723,0.643564,0.152019,0.301779,3.565736,...,1.842984,4.591365,5.010892,2.673438,1.751546,0.416211,-0.021706,5.495211,-2.173185,0
202013790112_R01C01,3.549663,-2.568295,3.376154,-2.423115,2.146287,2.130650,0.696030,0.014172,-0.046671,3.759265,...,2.204294,4.287095,5.069371,2.125303,1.331066,0.254350,-0.033212,4.786351,-2.640720,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202878330113_R04C01,4.068682,-2.615262,2.053700,-2.048839,2.720034,2.082817,0.738916,-0.027239,0.029772,4.206833,...,1.906852,3.914902,4.962997,2.868262,2.207983,0.779436,-0.237913,4.856758,-2.335723,1
201959750123_R03C01,3.089680,-2.867233,1.812105,-2.384390,2.959038,1.991487,0.710367,0.245171,0.358773,4.016917,...,1.957210,4.349414,4.931962,2.373272,2.632968,0.416159,0.002128,4.819631,-1.325193,1
201980430093_R08C01,4.213825,-2.562396,2.000722,-2.273870,2.961984,2.493231,0.876843,-0.080069,0.210484,4.590889,...,1.992776,4.178355,5.150082,2.854601,2.448002,0.615261,-0.091837,4.638254,-2.187200,1


In [32]:
# Export test df as csv
test.to_csv('Endometriosis_subset_test_feature_extract.csv')